In [6]:
from keras import layers
import numpy as np
from keras.models import Model
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding


def model(input_matrix_1, input_matrix_2, label, input_matrix_1_test, input_matrix_2_test, label_test):

    """Multiple Input Model with an Embedding Layer which follows by a Dense Layer and a Convolution Layer at each branch"""



    "First Branch for First Sequence"
    input_1 = layers.Input(
        shape=( np.shape(input_matrix_1)[1]))

    emb1 = layers.Embedding(530 , 10, input_length = 200)(input_1)          #embedding layer (input size = 1*200, output size = 10*200)
    emb1 = layers.Reshape((200, 10, 1))(emb1)

    cnn1 = layers.Conv2D(filters=16, kernel_size=(10, 2), activation = 'relu')(emb1)
    cnn1 = layers.MaxPooling2D(pool_size=(3, 1))(cnn1)
    cnn1 = layers.Flatten()(cnn1)


    "Second Branch for Second Sequence"
    input_2 = layers.Input(
        shape=( np.shape(input_matrix_2)[1]))
    
    emb2 = Embedding(530 , 10, input_length = 200)(input_2)
    emb2 = layers.Reshape((200, 10, 1))(emb2)
    cnn2 = layers.Conv2D(filters=16, kernel_size=(10, 2), activation = 'relu')(emb2)
    cnn2 = layers.MaxPooling2D(pool_size=(3, 1))(cnn2)
    cnn2 = layers.Flatten()(cnn2)


    "Use Add Layer for merging feature vectors given from each branch"
    merged_vector = layers.Add()([cnn1, cnn2])

    fc = layers.Dense(128, activation='sigmoid')(merged_vector)
    fc = layers.Dense(32, activation='relu')(fc)

    out_put = layers.Dense(2, activation='softmax')(fc)             #binary classification task


    model = Model(inputs=[input_1, input_2], outputs=[out_put])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit([input_matrix_1, input_matrix_2], label, batch_size= 64, epochs=35, verbose=1)

    print(model.summary())

    score = model.evaluate([input_matrix_1_test, input_matrix_2_test], label_test, verbose=1)

    return score[1]

In [7]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('shuffled_final_splash_df').sheet1


# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame(rows)
df.columns = df.iloc[0]
df = df.iloc[1:]

In [8]:
def make_input_data_list(dict_):
  seq1_ = list(dict_['seq1'])
  seq2_ = list(dict_['seq2'])
  seq1 = []
  for i in seq1_:
    seq = []
    for j in i:
      if (j == 'A'):
        seq.append(529)
      elif (j == 'U'):
        seq.append(361)

      elif (j == 'C'):
        seq.append(121)

      elif (j == 'G'):
        seq.append(169)

      else:
        seq.append(289)

    seq1.append(seq)
  seq2 = []
  for i in seq2_:
    seq = []
    for j in i:
      if (j == 'A'):
        seq.append(529)

      elif (j == 'U'):
        seq.append(361)

      elif (j == 'C'):
        seq.append(121)

      elif (j == 'G'):
        seq.append(169)

      else:
        seq.append(289)

    seq2.append(seq)
  seq1 = np.array(seq1)
  seq2 = np.array(seq2)
  return seq1, seq2

In [9]:
from sklearn.model_selection import KFold

"5 Fold Cross Validation"
kfold = KFold(5, True, 1)
acc = 0
for train, test in kfold.split(df):
  train_df = df.iloc[train, :]
  test_df = df.iloc[test, :]
  seq1_test = make_input_data_list(test_df)[0]
  seq2_test = make_input_data_list(test_df)[1]
  seq1_train = make_input_data_list(train_df)[0]
  seq2_train = make_input_data_list(train_df)[1]
  y_test = np.array(test_df[['tag1', 'tag2']].values.astype(int).tolist())
  y_train = np.array(train_df[['tag1', 'tag2']].values.astype(int).tolist())
  acc += model(seq1_train, seq2_train, y_train, seq1_test, seq2_test, y_test)

acc /= 5
print(acc)

Epoch 1/35
10/10 [==============================] - 1s 125ms/step - loss: 0.7033 - accuracy: 0.5283
Epoch 2/35
10/10 [==============================] - 1s 122ms/step - loss: 0.6835 - accuracy: 0.5477
Epoch 3/35
10/10 [==============================] - 1s 123ms/step - loss: 0.6679 - accuracy: 0.5460
Epoch 4/35
10/10 [==============================] - 1s 125ms/step - loss: 0.6418 - accuracy: 0.6511
Epoch 5/35
10/10 [==============================] - 1s 125ms/step - loss: 0.5873 - accuracy: 0.7803
Epoch 6/35
10/10 [==============================] - 1s 123ms/step - loss: 0.5068 - accuracy: 0.8449
Epoch 7/35
10/10 [==============================] - 1s 123ms/step - loss: 0.3970 - accuracy: 0.9063
Epoch 8/35
10/10 [==============================] - 1s 124ms/step - loss: 0.3021 - accuracy: 0.9128
Epoch 9/35
10/10 [==============================] - 1s 123ms/step - loss: 0.2036 - accuracy: 0.9451
Epoch 10/35
10/10 [==============================] - 1s 124ms/step - loss: 0.1373 - accuracy: 0.9661